In [1]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from read_data import get_matches_from_dir

/tmp/ipykernel_1006/505530613.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import read_data # weird notebook workaround
notebook_pardir = os.path.dirname(read_data.__file__)
notebook_pardir

'/home/daniel/Tekken8-Match-Stats'

In [3]:
start_date = "01_01_24"
end_date = None
df, failure_count = get_matches_from_dir(start_date=start_date, end_date=end_date)
print(f"{round((failure_count / len(df)) * 100, 2)}% of files failed to read")
print(df.columns)
df.to_csv(os.path.join(notebook_pardir, "plots", "data", "raw_matches.csv"), index=False)
df.head()

0.15% of files failed to read
Index(['battleId', 'battleType', 'gameVersion', 'winResult', 'totalRoundNum',
       'battleAt', 'viewNum', 'stageId', 'highlightFlag', '1pUserId',
       '1pPlayerName', '1pPolarisId', '1pOnlineId', '1pNgWordFlag',
       '1pPlatform', '1pRank', '1pTekkenPower', '1pCharaId', '1pWinRoundNum',
       '1pTagType01', '1pTagType02', '1pTagType03', '2pUserId', '2pPlayerName',
       '2pPolarisId', '2pOnlineId', '2pNgWordFlag', '2pPlatform', '2pRank',
       '2pTekkenPower', '2pCharaId', '2pWinRoundNum', '2pTagType01',
       '2pTagType02', '2pTagType03'],
      dtype='object')


,battleId,battleType,gameVersion,winResult,totalRoundNum,battleAt,viewNum,stageId,highlightFlag,1pUserId,...,2pOnlineId,2pNgWordFlag,2pPlatform,2pRank,2pTekkenPower,2pCharaId,2pWinRoundNum,2pTagType01,2pTagType02,2pTagType03
5994,006cfb1566bb4d2594d06a211c14d0d2,1,10201,1,3,1709288074,0,stg_0201,False,273631240126193946,...,D4RK_AN1MUS,0,8,6,94017,chr_0021,0,0,0,0
5995,007cf45ba31b4f2e9f0d279f6fb3e394,2,10201,1,5,1709288096,0,stg_1600,False,637185240126171527,...,suadongbin,0,8,15,110955,chr_0008,2,0,0,0
5996,00e0187ee62d4598940b3367a6341c7c,2,10104,2,4,1709288085,0,stg_1500,False,167050240125230443,...,F8TE2908,0,8,9,112605,chr_0006,3,0,0,0
5997,00ef53235548426fb37806058426c8f6,2,10201,2,5,1709288087,0,stg_0400,False,735222220610064832,...,76561198122207403,0,3,15,116528,chr_0009,3,0,0,0
5998,00fd1abc524141869e11c97a1b59709b,4,10201,1,3,1709288069,0,stg_1600,False,601958191106090458,...,76561198160266149,0,3,11,70784,chr_0012,0,0,0,0


In [4]:
cols_1p_normalized = ["battleId", "battleType", "gameVersion", "winResult", "totalRoundNum", "battleAt", "stageId",
                        "1pUserId", "1pPlayerName", "1pPlatform", "1pRank", "1pTekkenPower", "1pCharaId", "1pWinRoundNum"]

def mirror_row(row):
    row["winResult"] = 1 if row["winResult"] == 2 else 2 # swap win if 2p win
    for feature in ["UserId", "PlayerName", "Platform", "Rank", "TekkenPower", "CharaId", "WinRoundNum"]:
        row[f"1p{feature}"] = row[f"2p{feature}"]
    return row[cols_1p_normalized]

df_1p_normalized = pd.concat([df[cols_1p_normalized], df.copy().apply(mirror_row, axis=1)])
df_1p_normalized.to_csv(os.path.join(notebook_pardir, "plots", "data", "matches_1p_normalized.csv"), index=False)
df_1p_normalized.head()

,battleId,battleType,gameVersion,winResult,totalRoundNum,battleAt,stageId,1pUserId,1pPlayerName,1pPlatform,1pRank,1pTekkenPower,1pCharaId,1pWinRoundNum
5994,006cfb1566bb4d2594d06a211c14d0d2,1,10201,1,3,1709288074,stg_0201,273631240126193946,unluckyyy,3,10,74275,chr_0002,3
5995,007cf45ba31b4f2e9f0d279f6fb3e394,2,10201,1,5,1709288096,stg_1600,637185240126171527,temtu,8,13,92189,chr_0029,3
5996,00e0187ee62d4598940b3367a6341c7c,2,10104,2,4,1709288085,stg_1500,167050240125230443,DudeMeister_za,8,11,122546,chr_0017,1
5997,00ef53235548426fb37806058426c8f6,2,10201,2,5,1709288087,stg_0400,735222220610064832,Jeeq,3,18,140768,chr_0002,2
5998,00fd1abc524141869e11c97a1b59709b,4,10201,1,3,1709288069,stg_1600,601958191106090458,Nah I'd Win,3,12,79034,chr_0002,3


In [5]:
player_info = defaultdict(dict)

for uid, matches in df_1p_normalized.groupby("1pUserId"):
    for char, charmatches in matches.groupby("1pCharaId"):
        most_recent_game = charmatches.loc[charmatches["battleAt"].idxmax()]

        player_info[uid+char].update({
            "user_id": uid,
            "last_seen": most_recent_game["battleAt"],
            "username": most_recent_game["1pPlayerName"],
            "platform": most_recent_game["1pPlatform"],
            "rank_id": most_recent_game["1pRank"],
            "tekken_power": most_recent_game["1pTekkenPower"],
            "character_id": most_recent_game["1pCharaId"]
        })

df_player = pd.DataFrame(player_info).transpose().reset_index(drop=True)
df_player.to_csv(os.path.join(notebook_pardir, "plots", "data", "player_info.csv"), index=False)
df_player.head()

,user_id,last_seen,username,platform,rank_id,tekken_power,character_id
0,000001220902102523,1708009638,Rukoraku,9,14,102716,chr_0033
1,000001220902102523,1708012375,Rukoraku,9,8,102716,chr_0035
2,000003240206072109,1709364735,CookthemanEggs,8,14,107224,chr_0006
3,000008201007222404,1709315410,OniEUW,3,14,112413,chr_0011
4,000017200602140341,1709355259,초장동불꽃주먹,3,15,115112,chr_0015


In [6]:
matchup_info = []

for char, chardata in df.groupby("1pCharaId"): # 1p of given character vs others
    for otherchar, matches in chardata.groupby("2pCharaId"):
        matches = matches[matches["1pCharaId"] != matches["2pCharaId"]] # remove mirrors
        n_games = len(matches)
        n_wins = len(matches[matches["winResult"] == 1])
        win_rate = n_wins / n_games if n_games != 0 else np.NaN
        avg_rounds = matches["totalRoundNum"].mean()
        matchup_info.append([char, otherchar, n_games, n_wins, win_rate, avg_rounds])

for char, chardata in df.groupby("2pCharaId"): # 2p of given character vs others
    for otherchar, matches in chardata.groupby("1pCharaId"):
        matches = matches[matches["1pCharaId"] != matches["2pCharaId"]] # remove mirrors
        n_games = len(matches)
        n_wins = len(matches[matches["winResult"] == 2])
        win_rate = n_wins / n_games if n_games != 0 else np.NaN
        avg_rounds = matches["totalRoundNum"].mean()
        matchup_info.append([char, otherchar, n_games, n_wins, win_rate, avg_rounds])
        
df_matchup = pd.DataFrame(matchup_info, columns=["char", "other_char", "n_games", "n_wins", "win_rate", "avg_rounds"])
# entries for each char are currently separate, groupby to join them
df_matchup = df_matchup.groupby(by=["char", "other_char"]).agg(
    {"n_games": "sum", "n_wins": "sum", "win_rate": "mean", "avg_rounds": "mean"}).reset_index()
df_matchup.to_csv(os.path.join(notebook_pardir, "plots", "data", f"matchup_info.csv"), index=False)